In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("/storage1/fs1/yinjie.tang/Active/Shawn_Xiao/Qwen1.5-14B-Chat",
    device_map="auto", torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/storage1/fs1/yinjie.tang/Active/Shawn_Xiao/Qwen1.5-14B-Chat")

#Qwen1.5-72B-Chat-GPTQ-Int4
#Qwen1.5-14B-Chat

Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# pip install PyMuPDF
import pandas as pd
import os
import torch
import fitz

def read_pdf_and_split_into_chunks(pdf_path, chunk_size=1000):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    
    # Split text into chunks of `chunk_size` words
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

def ask_questions(chunks, question, system_prompt):
    combined_response = []
    for chunk in chunks:
        prompt_text = question + " " + str(chunk)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt_text}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=5000
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        combined_response.append(response)
    return " ".join(combined_response)

# Directory containing PDF files
pdf_directory = os.getcwd()
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# Prepare DataFrame
df = pd.DataFrame(columns=['PDF Title', 'Combined Response'])

# Define your question and system prompt for question 2
questions = " "  # Placeholder for the actual question text if needed


system_prompts = "You are specialized for analyzing scientific paper abstracts, focusing on identifying entities causal relationships related to biological studies, such as performance, species, genes, methods of genetic engineering, enzymes, proteins, and bioprocess conditions (e.g., growth conditions). It outputs the identified causal relationships between entities in combination pairs. The output strictly follows the format: (Entity A, Entity B), (Entity C, Entity D) ... with no additional text."


for i, pdf_file in enumerate(pdf_files):
    pdf_path = os.path.join(pdf_directory, pdf_file)
    chunks = read_pdf_and_split_into_chunks(pdf_path)
    combined_response = ask_questions(chunks, questions, system_prompts)
    df = df.append({'PDF Title': pdf_file, 'Answer to Question 2': combined_response}, ignore_index=True)
    
    # Calculate and show the progress percentage
    progress = ((i + 1) / len(pdf_files)) * 100
    print(f"Processing '{pdf_file}': {progress:.2f}% completed")

import pandas as pd
import re

# Assuming `df` is your DataFrame

def sanitize_for_excel(value):
    """
    Sanitize a string value for Excel, removing illegal characters.
    """
    if isinstance(value, str):
        # Replace or remove illegal characters as necessary
        # Here, we simply remove them
        value = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', value)
    return value

# Apply the sanitization function to every string value in the DataFrame
for col in df.columns:
    df[col] = df[col].apply(sanitize_for_excel)

# Now, try saving to Excel again
import pandas as pd
import re

# Assuming `df` is your DataFrame

def sanitize_for_excel(value):
    """
    Sanitize a string value for Excel, removing illegal characters.
    """
    if isinstance(value, str):
        # Replace or remove illegal characters as necessary
        # Here, we simply remove them
        value = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', value)
    return value

# Apply the sanitization function to every string value in the DataFrame
for col in df.columns:
    df[col] = df[col].apply(sanitize_for_excel)

# Now, try saving to Excel again
output_file_path = 'updated(Qwen1.5 14b)Yarrowia_pdf_causal.xlsx'
df.to_excel(output_file_path, index=False)


Processing 'Importance of the methyl-citrate cycle on glycerol metabolism in the yeast Yarrowia lipolytica.pdf': 0.43% completed
Processing 'Remodelling metabolism for high-level resveratrol production in Yarrowia lipolytica.pdf': 0.85% completed
Processing 'Subunit composition of mitochondrial complex I from the yeast Yarrowia lipolytica.pdf': 1.28% completed
Processing 'Efficient homologous recombination with short length flanking fragments in Ku70 deficient Yarrowia lipolytica strains.pdf': 1.71% completed
Processing 'Engineering the Yeast Yarrowia lipolytica for Production of Polylactic Acid Homopolymer.pdf': 2.14% completed
Processing 'Food-related applications of Yarrowia lipolytica.pdf': 2.56% completed
Processing 'Production of plant natural products through engineered Yarrowia lipolytica.pdf': 2.99% completed
Processing 'Targeting pathway expression to subcellular organelles improves astaxanthin synthesis in Yarrowia lipolytica.pdf': 3.42% completed
Processing 'Yarrowia lipoly

KeyboardInterrupt: 